In [13]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, AveragePooling2D, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model

print(tf.__version__)
print(np.__version__)

2.6.0
1.21.4


In [14]:
def build_residual_block(x, filters, kernel_size=3, stride=1, conv_shortcut=False, is_50=False):
    shortcut = x

    if conv_shortcut:
        shortcut = Conv2D(filters, kernel_size=1, padding='same', strides=stride)(x)
        shortcut = BatchNormalization()(shortcut)

    x = Conv2D(filters, kernel_size, padding='same', strides=stride)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    

    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)

    x = Add()([x, shortcut]) # shortcut connection

    x = Activation('relu')(x)
    
    return x

In [15]:
# ResNet 모델 자체를 생성하는 함수입니다.
def build_resnet(input_shape, num_classes, is_50, is_plain):
    inputs = Input(shape=input_shape)

    # kernel_size = 7*7, stride 2 = 1개
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    # 3*3, stride 2 max pooling
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

    # block 1: 3개 = 2 * 3 = 6개
    for _ in range(3):
        x = build_residual_block(x, 64, is_50=is_50)

    # block 2: shortcut이포함된 2개 + 2 * 3 = 2 * 4 = 8개
    x = build_residual_block(x, 128, stride=2, conv_shortcut=True, is_50=is_50)
    for _ in range(3):
        x = build_residual_block(x, 128, is_50=is_50)

    # block 3: shortcut이포함된 2개 + 2 * 5개 = 2 * 6 = 12개
    x = build_residual_block(x, 256, stride=2, conv_shortcut=True, is_50=is_50)
    for _ in range(5):
        x = build_residual_block(x, 256, is_50=is_50)

    # block 4: shortcut이포함된 2개 + 2 * 2개 = 2 * 3 = 6개
    x = build_residual_block(x, 512, stride=2, conv_shortcut=True, is_50=is_50)
    for _ in range(2):
        x = build_residual_block(x, 512, is_50=is_50)

    x = GlobalAveragePooling2D()(x) # global averagepooling 사용
    x = Flatten()(x)
    outputs = Dense(num_classes, activation='softmax')(x) # 1개

    model = Model(inputs, outputs)
    return model

In [17]:
"""
Total params: 21,315,338
Trainable params: 21,298,314
Non-trainable params: 17,024
"""
input_shape = (32, 32, 3)
num_classes = 10
resnet_34 = build_resnet(input_shape, num_classes, is_50=False, is_plain=False)
resnet_34.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_114 (Conv2D)             (None, 16, 16, 64)   9472        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_114 (BatchN (None, 16, 16, 64)   256         conv2d_114[0][0]                 
__________________________________________________________________________________________________
activation_105 (Activation)     (None, 16, 16, 64)   0           batch_normalization_114[0][0]    
____________________________________________________________________________________________